<a href="https://colab.research.google.com/github/96Asch/cv-bicycle-detection/blob/build_colab/CV_Bike_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bike detection using a Mask RCNN pretrained model
In this notebook we fine tune a Mask RCNN model, trained on the MSCOCO dataset, on our own created bike dataset. This dataset has already been pre-generated and is stored on a publicly accesible drive.

You can run the notebook in sequence except for when downgrading Keras. More instruction on this will be below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Install required packages

Here we clone the Mask RCNN repo and install the requirements from the given requirements.txt file

In [ ]:
%cd
  
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

In [ ]:
%cd ~/Mask_RCNN

!pip install -q PyDrive
!pip install -r requirements.txt
!pip install 'h5py<3.0.0'
!python setup.py install

#Download and extract dataset
This downloads the COCO_Bikes dataset from Google Drive and extracts it into Mask_RCNN/dataset/

In [ ]:
%cd ~/Mask_RCNN


fileId = '14EAGfdRPSuIcrB3nmvXOfjtICFNALnMT'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

if not os.path.exists('./dataset'):
  os.makedirs('dataset')
os.chdir('dataset')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('./COCO_Bikes'):
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)


# Download the pretrained weights
We fetch the MSCOCO pretrained weights of Mask RCNN


In [ ]:
%cd ~

modelId = '1Mos_U0jPpmNleZuLzGSTRTp7Ct4-PdAK'

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('./pretrained.h5'):
  downloaded = drive.CreateFile({'id': modelId})
  downloaded.GetContentFile('pretrained.h5')

#Configure the the Config and Dataset 

The Config class handles the configuration of the training.
The Dataset class handles the import of our custom dataset

In [ ]:
%cd ~/Mask_RCNN
%tensorflow_version 1.x
import os
import argparse
import tensorflow as tf
import datetime
import warnings

import numpy as np
from pycocotools.coco import COCO
import sys


from mrcnn.config import Config
from mrcnn import utils

sys.path.append(os.path.join("/root/Mask_RCNN", "samples/coco/"))  # To find local version
import coco

class DelftBikesConfig(Config):
    NAME = "delft-bikes"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # background + bike
    STEPS_PER_EPOCH = 100
    LEARNING_RATE = 0.0001
  

class COCODelftBikes(coco.CocoDataset):

    def load(self, dataset_dir: str, subset: str) -> None:

        path = os.path.join(dataset_dir, subset)
        annotation = os.path.join(dataset_dir, f"{subset}.json")

        coco = COCO(annotation)
        class_ids = sorted(coco.getCatIds())
        
        image_ids = []
        for id in class_ids:
            image_ids.extend(list(coco.getImgIds(catIds=[id])))
        # Remove duplicates
        image_ids = list(set(image_ids))

        self.add_class("coco", 1, coco.loadCats(1)[0]["name"])
        
        
          # Add images
        for i in image_ids:
            self.add_image(
                "coco", image_id=i,
                path=os.path.join(path, coco.imgs[i]['file_name']),
                width=coco.imgs[i]["width"],
                height=coco.imgs[i]["height"],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))

    def load_mask(self, image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "coco":
            return super(coco.CocoDataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = annotation['category_id']
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            return mask, np.ones([mask.shape[-1]], dtype=np.uint8)
        else:
            # Call super class to return an empty mask
            return super(coco.CocoDataset, self).load_mask(image_id)

###  Check if the GPU is running

In [ ]:
%tensorflow_version 1.x



import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Train the model
Here we begin training the model, by first enabling Tensorboard in order to visualize each loss later on.

In [ ]:
%load_ext tensorboard

#### Re-install the correct version of Keras
For some reason, the built-in version of Keras gets reset when using the tensorflow version command. So we install the correct version: 2.2.5.
You must reset the runtime to change the versions. Then re-run the config and dataset cell before continuing.

In [ ]:
!pip install keras==2.2.5
!pip list | grep Keras

We load the pretrained model and begin training using the dataset and config files specified before

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import mrcnn.model as modellib

config = DelftBikesConfig()
model_save_dir = os.path.join("/content", "exps")
pretrained_model_file = os.path.join("/root", "pretrained.h5")
data_set_dir = os.path.join("/root", "Mask_RCNN", "dataset", "COCO_Bikes")

with tf.device(device_name):
    model = modellib.MaskRCNN(mode='training', config=config, model_dir=model_save_dir)

assert os.path.exists(pretrained_model_file)
    
model.load_weights(pretrained_model_file, by_name=True, exclude=[ "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

dataset_train = COCODelftBikes()
dataset_train.load(data_set_dir, 'train')
dataset_train.prepare()

dataset_val = COCODelftBikes()
dataset_val.load(data_set_dir, 'test')
dataset_val.prepare()

model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=100,
            layers='all')

# Visualize the losses using TensorBoard
You should edit the path after --logdir and put the path to folder created by Mask RCNN which contains the tfevents file

In [ ]:
%tensorboard --logdir /root/exps/delft-bikes20220615T2029